# Introduction to Tensorflow

In [1]:
import tensorflow as tf
import numpy as np

## 1. Tensor
Please read the tutorial in the following link first:
https://www.tensorflow.org/guide/tensors

Now let's review the tutorial by coding it here. Feel free to change the code cells and inspect the changes in the output.

### 1.1. Creating Tensors

When writing Tensorflow programs the main object you manipulate and pass is `tf.Tensor`. Some types of tensors are special:
* `tf.constant`

* `tf.Variable`

* `tf.placeholer`

In [12]:
x = tf.constant([1,2,3,4,5], name='x')
y = tf.constant([[1,2,3,4,5],[2,0,3,4,2]], name='y')
v = tf.Variable([1., 2.33, 3.11], dtype=tf.float32, name='v')
s = tf.Variable(['Hello!'], dtype=tf.string, name='s')
p = tf.placeholder(name='p', shape=[None, 10], dtype=tf.float32) # number of elements in the first dimension will be determined when we run a session. 
print(x)
print(y)
print(v)
print(s)
print(p)
print('\n')
print(type(x))
print(type(v))
print(type(s))
print(type(p))

### 1.2. Shape and Rank

In [13]:
z = tf.zeros([1, 2, 3], dtype=tf.int32)
print(z.shape)  # What do you expect to get as the ouput?

In [14]:
print(type(z.shape))  # What do you expect to get as the ouput?

In [15]:
var1 = tf.Variable(initial_value=[[2, 3]], dtype=tf.int32)
r = tf.rank(var1)
print(r) # What do you expect to get as the ouput?

### 1.3. Referring to tf.Tensor slices

In [16]:
# rank 1
vector = tf.constant([1, 2, 3, 4])
x = vector[1]
y = vector[0:2]
i = tf.constant(2)
print(x)
print(y)
print(vector[i])

print('\n')

# rank 2
mat = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
z = mat[2]
w = mat[0:2, 1]
t = mat[:, 0:2]
print(z)
print(w)
print(t)

### 1.4. Reshape

In [17]:
a = tf.ones([2, 3, 4])
b = tf.reshape(a, [4, 6])
c = tf.reshape(a, [-1])

print(a)
print(b)
print(c)

### 1.5. Cast

In [18]:
a = tf.constant([1, 2, 6])  # Can you guess the datatype?
b = tf.cast(a, dtype=tf.float64)
print(a)
print(b)

## 2. Graph

TensorFlow uses a dataflow graph to represent your computation in terms of the dependencies between individual operations. For example, in a Tensorflow graph, the `tf.matmul` operation would correspond to a single node with two incoming edges (the matrices to be multiplied) and one outgoing edge. <br/>
A `tf.Graph` contains two relevant kinds of information:
* **Graph structure**: The nodes and edges of the graph, indicating how individual operations are composed together.
* **Graph collections**: Tensorflow provides a general mechanism for storing collections of metadata in `tf.Graph`. (collections are out of the scope of this tutorial, so don't bother yourself if it seems confusing. But, you can read more about them [here](https://www.tensorflow.org/guide/graphs) if you'd like.)<br/>

Most Tensorflow programs start with a dataflow graph construction phase. In this phase, you invoke Tensorflow API functions that constructs new `tf.Operation` (node) and `tf.Tensor` (edge) objects and add them to a `tf.Graph` instance. 
Tensorflow provides a **default graph** that is an implicit argument to all functions in the same context. For example:
* Calling `tf.constant(42.0)` creates a single `tf.Operation` that produces the value 42.0, adds it to the default graph, and returns a `tf.Tensor` that represents the value of the constant.
* Calling `tf.matmul(x, y)` creates a single `tf.Operation` that multiplies the values of `tf.Tensor` objects `x` and `y`, adds it to the default graph, and returns a `tf.Tensor` that represents the result of the multiplication.

For example, consider the following code. It builds a computational graph (which is the default graph) with 3 nodes: `x`, `y`, `z`.

In [6]:
# building the graph...
x = tf.constant(3)
y = tf.constant(4)
z = x * y
# 3 nodes have been added to the graph by far

Eech Tensor object resides in a graph. By running the following code snippet you will see that `x`, `y`, `z` are all in the default graph.

In [17]:
print(x.graph is tf.get_default_graph())
print(y.graph is tf.get_default_graph())
print(z.graph is tf.get_default_graph())

You can clear the nodes in the default graph and reset it by using `tf.reset_default_graph`.

In [19]:
x = tf.constant(3) 
print(x.graph is tf.get_default_graph())
tf.reset_default_graph()
print(x.graph is tf.get_default_graph())

## 3. Session

Once the computational graph is built, you can run the computation that produces a particular `tf.Tensor` and fetch the values assigned to it. With `tf.Session()` you can run a computational graph. We usually use a session with a context manager in Python, So we don't need to worry about closing the session. you can read more about context managers in Python [here](https://en.m.wikibooks.org/wiki/Python_Programming/Context_Managers) if you'd like.

In [20]:
# building the computational graph...
# in this example the computational graph is so simple and contains only one tf.constant
x = tf.constant([1, 2, 3, 4, 5], name ='x')

# running the computational graph with a session
with tf.Session() as sess:
    x_np = sess.run(x)  # every tensor after runing session on it would give us a numpy array
    
print(type(x_np))
print(x_np)

In [21]:
# A more sophisticated example
# in this example we use arithmetic operations too

# building the computational graph...
x = tf.constant([1, 2, 3, 4], dtype=tf.float32)
y = x ** 2  # The overloaded operators are available in Tensorflow
z = y - 1
print(z)

# running the computational graph with a session
with tf.Session() as sess:
    z_np = sess.run(z)
    print(z_np)  # What output do you expect to get?

Obviously, We don't want to work only with constant tensors. In fact there should be a way to feed the data into our computational model. This can be done by using `tf.placeholder`.

In [22]:
# building the computational graph...
# we will feed the data into x when we run the graph
x = tf.placeholder(name='x', shape=[2, 3], dtype=tf.float32)
y = x / 2

# running the computational graph with a session
with tf.Session() as sess:
    x_np = np.random.randn(2, 3)
    y_out = sess.run(y, feed_dict={x: x_np})  # feeding the data to feeddict parameter of sess.run() as a Python dictionary
print(y_out)

There is no need to specify the number of elements in each dimension of a placeholder. We will use this nice property later when we feed mini-batches of data with not necessarily a fixed size to our model.

In [23]:
x = tf.placeholder(name='x', shape=[None, 3], dtype=tf.float32)

x_np_1 = np.random.randn(4, 3)
x_np_2 = np.random.randn(2, 3)

with tf.Session() as sess:
    x_out1 = sess.run(x, feed_dict={x: x_np_1})
    x_out2 = sess.run(x, feed_dict={x: x_np_2})
    print('x_out1:\n', x_out1)
    print('x_out2:\n', x_out2)

As you witnessed above, you can run a computational graph many times (in this case the computational graph was consisted of only one placeholder), each time with a different input. Thus, you build the graph once but you can run it many times.

You can also evaluate the value of a tensor with `Tensor.eval()` method. The eval method only works when a default session is active. `Tensor.eval()` returns a numpy array with the same contents as the tensor.

In [30]:
a = tf.constant([5, 11, 22, 0], dtype=tf.float32)


# creating a session for evaluating a
# if you run tf.eval() without creating session you will get an error
with tf.Session() as sess:
    print(a.eval())
    print(type(a.eval()))

## 4. Supplementary notes

### Variable

A `tf.Variable` represents a tensor whose value can be changed by running ops on it. The `Variable()` constructor requires an intitial value for the variable, which can be a Tensor of any type and shape. The initial value defines the type and shape of the variable. After construction, the type and shape of the variable are fixed. The value can be changed using one of the assign methods. 

In [24]:
v = tf.Variable(initial_value=[2,3,8], name='v')
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)  # initialize variables
    v_out = sess.run(v)
print(v_out)

Note that you have to run the initialization of variables residing inside your computational graph before running any other operations on them. Otherwise, you will get an error! The operation init above does this initialization when we run sess on it. So don't forget: **All variables needs to be initialized!**

In [25]:
v = tf.Variable(initial_value=tf.zeros([4, 5]), dtype=tf.float32)
v = v + 1  # you can normaly treat with variables like any other tensor in computations

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())  # a shorter way of initializing variables
    v_out = sess.run(v)
    print(v_out)

In [26]:
w = tf.Variable(initial_value=tf.zeros([4, 5]), dtype=tf.float32)
t = tf.Variable(initial_value=tf.ones([4, 5]), dtype=tf.float32)
with tf.Session() as sess:
    sess.run(w.initializer)  # another way of initializing a variable
    t.initializer.run()  # another way of initializing a variable
    w_np = sess.run(w)
    t_np = sess.run(t)
    print(w_np)
    print(t_np)

Although you can create variables with the `tf.Variable()` constructor, the preferred way of creating variables is using `tf.get_variable()`. This function requires you to specify the variable's name. It also allows you to reuse a previously created variable of the same name (don't worry if it seems confusing! you will work more with this function in your next assignments).

In [27]:
# close the default graph stack and resets the global default graph
tf.reset_default_graph()

var1 = tf.get_variable(name='var1', shape=[1, 2, 3], initializer=tf.zeros_initializer)
var2 = tf.get_variable(name='var2', shape=[1, 2, 3], initializer=tf.random_normal_initializer)

print(var1)
print(var2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    var1_np, var2_np = sess.run([var1, var2])  # running a session on multiple tensors at once

To assign a value to a variable, you can use the methods `assign`, `assign_add`, `assign_sub`, ... . For example:

In [28]:
tf.reset_default_graph()

var3 = tf.get_variable('var3', shape=(1, 2), initializer=tf.zeros_initializer())
assignment_operation = var3.assign_add(tf.ones([1, 2]))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output = sess.run(assignment_operation)
    print(output, '\n')
    
    var3_np = sess.run(var3)
    print(var3_np, '\n')
    
    var3_np_1 = sess.run(var3.read_value())  # reading the value of a variable
    print(var3_np_1)

Here is the link to a tutorial for reading more about variables:
https://www.tensorflow.org/guide/variables

### name scope

Tensorflow uses namespaces to organize tensors/variables and operations. An example of tensor name is `scope_outer/scope_inner/tensor_a:0`. `tensor_a` is the actual name of the tensor. The suffix `:0` is the endpoint used to give the tensors returned from an operation unique identifiers, i.e. `:0` is the first tensor, `:1` is the second and so on.

Some medium or high level APIs like Keras will handle scope naming for you. But if you want to program in low-level in Tensorflow you have to do this manually. This is usually done by using `tf.name_scope` or `tf.variable_scope`. [This link](https://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow/43581502#43581502) gives a great explanation to the differences between these two. It turns out there is only one difference, `tf.variable_scope` affects `tf.get_variable` while `tf.name_scope` does not. `tf.variable_scope` also has a parameter `reuse` which allow you to reuse the same variable (with the same name in the same namespace) in the different part of the code without having to pass a reference to that variable around. Usually you would want to use `variable_scope` unless there is a need to put operations and variables in different levels of namespaces.

Don't worry if you are confused. The goal of this notebook is just familiarizing you with Tensorflow and we are not aiming to work with namescopes in this assignment. You will learn exactly how to work with them in your next assignments. 

In [29]:
tf.reset_default_graph()

with tf.name_scope('scope1'):
    x = tf.constant(2, name = 'y')
    y = tf.constant(2, name = 'y')
    c = tf.constant(2, name = 'w')

print(x.name)
print(y.name)
print(c.name)
print('\n')

with tf.name_scope('scope2'):
    c1 = tf.constant(2,name = 'w')
    c2 = tf.constant(2,name = 'w1')

print(c1.name)
print(c2.name)
print('\n')

with tf.name_scope('scope3'):
    d1 = tf.constant(2,name = 'w')
    d2 = tf.constant(2,name = 'w1')
    w2 = tf.get_variable(name='w2', shape=[1, 2], dtype=tf.float32)

print(d1.name)
print(d2.name)
print(w2.name)
print('\n')

with tf.variable_scope('scope4'):
    v = tf.Variable(initial_value=[1., 2.], name='v')
    v1 = tf.Variable(initial_value=[1., 2.], name='v')
    v2 = tf.get_variable(name='v2', initializer=tf.zeros_initializer, shape=[1, 2], dtype=tf.float32)

print(v.name)
print(v1.name)
print(v2.name)

### Evaluating Tensor attributes

As it was mentioned in the beginning of notebook, **rank** and **shape** are two attributes of tensors. We can run a session on `tf.rank` and `tf.shape` to get these attributes with a nice format.

In [31]:
a_0 = tf.constant(np.random.randint(0 , 5, ()), dtype=tf.float64)  # tensor with rank 0, scalar
a_1 = tf.constant(np.random.randint(0 , 5, (6)), dtype=tf.float64)  # tensor with rank 1, vector
a_2 = tf.constant(np.random.randint(0 , 5, (3,6)), dtype=tf.float64)  # tensor with rank 2, matrix


a_0_rank = tf.rank(a_0)   # return 0d-Tensor, type int32 ,scaler
a_1_rank = tf.rank(a_1)    
a_2_rank = tf.rank(a_2)    


with tf.Session() as sess:
    a,b,c = sess.run([a_0_rank, a_1_rank, a_2_rank])
    
print(a,b,c)

In [32]:
a = tf.truncated_normal((2,4,5))  # tf.truncated_normal is a random initializer in Tensorflow

shape = tf.shape(a)  # 1d-tensor containing shape of a

with tf.Session() as sess:
    shape_r = sess.run(shape)
    
print(shape_r)

### Useful mathematical functions 

* `tf.reduce_sum`

* `tf.reduce_mean`

In [18]:
tf.reset_default_graph()

x = tf.get_variable(name='x', initializer=[[1, 1, 1], [1, 1, 1]])
y = tf.get_variable(name='y', initializer=[[1, 3], [1, 3], [1, 3]])
a = tf.matmul(x, y)
s = tf.reduce_sum(a)
s0 = tf.reduce_sum(a, axis=1)
s1 = tf.reduce_sum(a, axis=0)
m = tf.reduce_mean(a)
m0 = tf.reduce_mean(a, axis=0)
m1 = tf.reduce_mean(a, axis=1, keepdims=True)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x_np, y_np, a_np, s_np, s0_np, s1_np, m_np, m0_np, m1_np = sess.run([x, y, a, s, s0, s1, m, m0, m1])

print('x_np:\n', x_np)
print('y_np:\n', y_np)
print('a_np:\n', a_np)
print('s_np:\n', s_np)
print('s0_np:\n', s0_np)
print('s1_np:\n', s1_np)
print('m_np:\n', m_np)
print('m0_np:\n', m0_np)
print('m1_np:\n', m1_np)